测试线性混合效应模型。

In [1]:
# 设置因子水平数
n1 <- 2  # g1
n2 <- 3  # g2

# 创建因子索引
i <- gl(n1, n2)
j <- gl(n2, 1, n1 * n2)

# 随机效应
b1x <- rnorm(n1, mean = 0, sd = 1)
b2x <- rnorm(n2, mean = 0, sd = 2)

# 初始数据框
dt0 <- data.frame(i = i, j = j)

# 生成最终数据框
dtc <- within(dt0, {
  eps <- rnorm(nrow(dt0), mean = 0, sd = 0.2)
  b1 <- b1x[i]
  b2 <- b2x[j]
  y <- 10 + b1 + b2 + eps
  g1 <- factor(LETTERS[i])
  g2 <- factor(j, labels = letters[1:n2])
})

dtc

i,j,g2,g1,y,b2,b1,eps
<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,a,A,10.251368,2.071718,-2.233387,0.41303677
1,2,b,A,6.262446,-1.992034,-2.233387,0.48786752
1,3,c,A,3.194801,-4.519488,-2.233387,-0.05232492
2,1,a,B,10.719548,2.071718,-1.381016,0.02884648
2,2,b,B,6.779392,-1.992034,-1.381016,0.15244283
2,3,c,B,3.973049,-4.519488,-1.381016,-0.12644697


In [38]:
library(lme4.0)
fit <- lmer(y ~ 1 + (1 | g1) + (1 | g2), data = dtc)
summary(fit)

ERROR: Error in library(lme4.0): there is no package called 'lme4.0'


In [ ]:
library(nlmeU)
library(nlme)
# 使用nlmeU包进行模型拟合
fit <- lme(y ~ 1, random = ~ 1 | g1, data = dtc)
logLik1(fit, dt1 = dtc)

[1] -14.83181

In [26]:
library(car)
cooks.distance(fit)

1         2         3         4         5         6 
1.2166233 0.3116569 4.5971898 1.0349257 0.3207704 4.2704880

In [47]:
library(lmerTest)
# 进行检验
fit <- lmer(y ~ 1 + (1 | g1) + (1 | g2), data = dtc)
summary(fit)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: y ~ 1 + (1 | g1) + (1 | g2)
   Data: dtc

REML criterion at convergence: 13.1

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-0.89069 -0.37577 -0.00168  0.40163  0.85846 

Random effects:
 Groups   Name        Variance Std.Dev.
 g2       (Intercept) 11.9888  3.4625  
 g1       (Intercept)  0.1681  0.4100  
 Residual              0.0139  0.1179  
Number of obs: 6, groups:  g2, 3; g1, 2

Fixed effects:
            Estimate Std. Error    df t value Pr(>|t|)  
(Intercept)    6.863      2.021 2.083   3.397   0.0726 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

In [49]:
confint(intervals)

ERROR: Error in object$coefficients: object of type 'closure' is not subsettable


In [37]:
library(tidyverse)
anova(fit) %>% print()

Type III Analysis of Variance Table with Satterthwaite's method
     Sum Sq Mean Sq NumDF DenDF F value Pr(>F)


In [39]:
# 安装（如果尚未安装）
# install.packages("pbkrtest")

library(pbkrtest)

# 注意：为了进行比较，模型必须是用lmer()拟合的
# full_model 和 null_model 来自上一步，但用REML拟合也可以
full_model_reml <- lmer(Reaction ~ Days + (1 + Days | Subject), data = sleepstudy)
null_model_reml <- lmer(Reaction ~ 1 + (1 + Days | Subject), data = sleepstudy)


# 方法A: Kenward-Roger F检验 (一种改进的F检验近似)
KR_test <- KRmodcomp(full_model_reml, null_model_reml)
summary(KR_test)

# 方法B: 参数化自举法 (Parametric Bootstrap)
# nsim设置了模拟次数，实际应用中建议至少1000次，这里为了速度设为200
PB_test <- PBmodcomp(full_model_reml, null_model_reml, nsim = 200)
summary(PB_test)

F-test with Kenward-Roger approximation; time: 0.05 sec
large : Reaction ~ Days + (1 + Days | Subject)
small : Reaction ~ 1 + (1 + Days | Subject)
         stat    ndf    ddf F.scaling   p.value    
Ftest  45.853  1.000 17.000         1 3.264e-06 ***
FtestU 45.853  1.000 17.000           3.264e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00667686 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00616354 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00249764 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00206685 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.0031531 (tol = 0.002, component 1)"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00329668 (tol = 0.002, c

Bootstrap test; time: 2.68 sec; samples: 200; extremes: 0;
large : Reaction ~ Days + (1 + Days | Subject)
Reaction ~ 1 + (1 + Days | Subject)
           stat     df    ddf   p.value    
LRT      23.537  1.000        1.226e-06 ***
PBtest   23.537                0.004975 ** 
Gamma    23.537               1.763e-05 ***
Bartlett 21.414  1.000        3.701e-06 ***
F        23.537  1.000 22.175 7.402e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [40]:
# 完整模型 (包含Days)，使用ML估计
full_model <- lmer(Reaction ~ Days + (1 + Days | Subject), data = sleepstudy, REML = FALSE)

# 零模型 (不包含Days)，也使用ML估计
null_model <- lmer(Reaction ~ 1 + (1 + Days | Subject), data = sleepstudy, REML = FALSE)

# 使用anova()函数比较两个模型
anova(null_model, full_model)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
null_model,5,1785.476,1801.441,-887.7379,1775.476,NA,NA,NA
full_model,6,1763.939,1783.097,-875.9697,1751.939,23.53654,1,1.22564e-06


In [46]:
library(lme4)
library(lmerTest) # 加载lmerTest以便summary能显示p值，但对本例的LR检验无直接影响

# 完整模型：随机截距 + 随机斜率
full_model <- lmer(Reaction ~ Days + (1 + Days | Subject), data = sleepstudy, REML = TRUE)

# 简化模型：只有随机截距
reduced_model <- lmer(Reaction ~ Days + (1 | Subject), data = sleepstudy, REML = TRUE)

# 安装（如果尚未安装）
# install.packages("RLRsim")
library(RLRsim)

# 使用 exactRLRT 进行精确的限制性似然比检验
test_result <- anova(reduced_model, full_model)

print(test_result)

refitting model(s) with ML (instead of REML)



Data: sleepstudy
Models:
reduced_model: Reaction ~ Days + (1 | Subject)
full_model: Reaction ~ Days + (1 + Days | Subject)
              npar    AIC    BIC  logLik deviance  Chisq Df Pr(>Chisq)    
reduced_model    4 1802.1 1814.8 -897.04   1794.1                         
full_model       6 1763.9 1783.1 -875.97   1751.9 42.139  2  7.072e-10 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
